In [ ]:
# kaggleのAPIを使えるようにする
from google.colab import files
from IPython.display import clear_output

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

!kaggle competitions download -c h-and-m-personalized-fashion-recommendations
!unzip h-and-m-personalized-fashion-recommendations.zip
clear_output()

In [ ]:
# articlesにtransactionから商品単価を求めてくっつける
# 実行済
price_df = transactions.groupby('article_id')['price'].agg(['mean'])
new_articles = pd.merge(articles, price_df, how='left', on = 'article_id').rename(columns={'mean': 'price'})

In [ ]:
# 商品ごとに、顧客の世代を付与
# 実行済
def product_for_age(x):
  item_id = x
  result = dict()
  for i in range(len(who_were_buy_the_item)):
    # item_id = who_were_buy_the_item.index[i]
    try:
      customers = who_were_buy_the_item.loc[item_id, : ]
    except:
      return result
    item_customer_age = customer_buyitem[customer_buyitem.customer_id.isin(customers[0])].age
    min_age = item_customer_age.min()
    max_age = item_customer_age.max()
    if (np.isnan(min_age) == True) & (np.isnan(max_age) == True):
      # print(item_id)
      # print(f'購買人数：{len(customers[0])}')
      # print('年齢情報不明')
      # print("*" * 100)
      return result
    else:
      result["mean_age"] = item_customer_age.mean()
      result["median_age"] = item_customer_age.mean()
      result['min_age'] = min_age
      result['max_age'] = max_age
      return result

def sep_age(x):
  if len(x) == 0:
    pass
  else:
    return x['max_age']

articles['max_age'] = articles.about_age.progress_apply(lambda x: sep_age(x))
articles['about_age'] = articles.article_id.progress_apply(lambda x: product_for_age(x))

In [ ]:
# ユーザーが何色のどの種類を多く買う傾向にあるのか調査
# 実行済
def user_like_color(x):
  if isinstance(x, float):
    pass
  else:
    return articles[articles.article_id.isin(x)].groupby('colour_group_name').count()['article_id'].idxmax()

customer_buyitem['most_like_color'] = customer_buyitem.article_id.progress_apply(lambda x: user_like_color(x))

In [ ]:
# 過去の買い上げ履歴から男女を判定する
# 実行済
def sex_pred(x):
  if isinstance(x, float):
    pass
  else:
    prod_kind = articles[articles.article_id.isin(x)].groupby('index_group_name').count()['article_id'].idxmax()
    if (prod_kind == 'Baby/Children') or (prod_kind == 'Ladieswear'):
      return 'woman'
    elif prod_kind == 'Menswear':
      return 'man'
    elif prod_kind == 'Sport':
      return 'sport'
    else:
      return 'devided'

customer_buyitem['sex'] = customer_buyitem['article_id'].progress_apply(lambda x: sex_pred(x))
# customer_buyitem.to_csv("customer_buyitem.csv")

In [ ]:
# 商品に性別属性ラベルを割り振る
# 実行済
def item_sex(x):
  if isinstance(x, float):
    pass
  else:
    if (x == 'Ladieswear') or (x == 'Lingeries/Tights') or (x == 'Ladies Accessories'):
      return 'woman'
    elif (x == 'Menswear'):
      return 'man'
    elif (x == 'Sport'):
      return 'sport'
    elif (x == 'Divided'):
      return 'devided'
    else:
      return 'child'
      
articles['item_sex'] = articles.index_name.progress_apply(lambda x: item_sex(x))

In [ ]:
 # 過去の買い上げアイテムの平均購入単価から、ユーザーが購入する可能性のある商品価格帯を
# 標本平均＝平均価格の正規分布に従うとして95%信頼区間の上限、下限を求める
# 実行済
from scipy import stats
import numpy as np

def user_buy_price(x):
  result_dict = dict()
  if isinstance(x, list):
    data = x
    a = 0.95 #信頼水準。ここでは 95%。
    d = len(data)-1 #自由度 (= サンプルサイズ-1)
    m = np.mean(data) #標本平均
    s = stats.sem(data) #標準誤差。不偏分散から計算。
    # result = stats.t.interval(alpha = a, df = d, loc = m, scale =s)
    result = stats.norm.interval(alpha = a, loc = m, scale = np.std(data))
    # result[0]: 金額下限, result[1]: 金額上限
    result_dict['mix'] = result[0]
    result_dict['max'] = result[1]
    result_dict['mean'] = np.mean(data)
    return result_dict
  else:
    pass

customer_buyitem['user_buy_min_max_price'] = customer_buyitem.price.progress_apply(lambda x: user_buy_price(x))
# customer_buyitem.to_csv("customer_buyitem.csv")

  0%|          | 0/1371980 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:263: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims, where=where)
100%|██████████| 1371980/1371980 [10:51<00:00, 2104.89it/s]


# 方針メモ
・ユーザーが何を買ったのかを一覧にするcsv作成<br>
　→作成済み。<br>
・ユーザーがアクティブかそうじゃないかを分ける<br>
　→アクティブ度合いで、おすすめするものを変える<br>
　　例：低アクティブユーザーであれば、多くの類似の人が買っていて、買っていないものをレコメンド<br>
　　　　高アクティブユーザーであれば、過去の購買履歴をもとに季節の商品をレコメンド<br>
　→商品に、万人向けスコアみたいなものを導入<br>
　→商品に、どの季節で使う商品かのタグ付を行う（オールシーズン：0, 春：1、夏：2、秋：3、冬：4）<br>
・顧客の買い上げ履歴から、商品画像を吸い上げて、類似画像検索から上位○位までの類似商品をピックアップする<br>

・ユーザーごとに買わない人はこの先も買わないんじゃ？<br>
　→評価指標的には予測しない理由がないからムダ<br>

・商品ごとに、誰に買われているかをラベル付けする
　例：何歳の人に買われる事が多いのか。

# 評価されるポイントについて
・今回提出するデータは、2020-9-22までの購買データから、2022-9-23〜2022-9-29までの予測結果で評価される。<br>
　→夏、真冬ものは買われないのではないか？<br>

# 実行はここから

In [ ]:
# kaggleのAPIを使えるようにする
from google.colab import files
from IPython.display import clear_output

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

!kaggle competitions download -c h-and-m-personalized-fashion-recommendations
!unzip h-and-m-personalized-fashion-recommendations.zip
clear_output()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive


In [ ]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from datetime import datetime, date, timedelta

from collections import Counter, defaultdict
from PIL import Image
from pathlib import Path
from tqdm import tqdm
import ast
tqdm.pandas()

In [ ]:
# %%writefile create_buyitem_every_customers.py

# article.csvのcolumns
columns = ['article_id', 'item_sex', 'mean_age', 'median_age', 'min_age', 'max_age']

#  各csv読み込み
articles = pd.read_csv("articles.csv", dtype={"article_id": str})
customers = pd.read_csv("customers.csv", index_col='customer_id')
# customer_buyitem = pd.read_csv("customer_buyitem.csv", index_col='customer_id')
# to_csvにした時に文字列になったlistを元に戻す
# customer_buyitem['article_id'] = [ast.literal_eval(d) if type(d) != np.float else d for d in customer_buyitem['article_id']]
# customer_buyitem['price'] = [ast.literal_eval(d) if type(d) != np.float else d for d in customer_buyitem['price']]
# customer_buyitem['user_buy_min_max_price'] = [ast.literal_eval(d) if type(d) != np.float else d for d in customer_buyitem['user_buy_min_max_price']]
transactions = pd.read_csv("transactions_train.csv")

# transactionsの商品番号がint型なのと、最終的に文字列型かつ先頭に0がつくので最初にその補正
# article_hosei = lambda x: '0' + str(x)
# articles.article_id = articles.article_id.apply(article_hosei)
# transactions['article_id'] = transactions['article_id'] .apply(article_hosei)

# 購買履歴からユーザー毎に購入品とトータルで利用した金額を集計（累計買い上げ金額はあまり意味ないかも？）
# sample = transactions.groupby(['customer_id']).agg({'article_id': list, 'price': list})
# customer_buyitem = pd.merge(customers, sample, how='left', on='customer_id')

# customer_buyitem.to_csv("customer_buyitem.csv")

"""
transactionsに商品情報と顧客情報をマージ
"""
# transactions.drop('sales_channel_id', axis=1, inplace=True)
# customer_buyitem.drop(['FN', 'Active', 'club_member_status', 'fashion_news_frequency', 'postal_code', 'article_id', 'price', 'user_buy_min_max_price'], axis=1, inplace=True)
# customer_buyitem.rename(columns={'sex': 'customer_sex'}, inplace=True)
# a_df = pd.merge(transactions, customer_buyitem, how='left', on='customer_id')
# b_df = pd.merge(a_df, articles[columns], how='left', on='article_id')
# b_df.to_csv('merged_df.csv', index=False)

'\ntransactionsに商品情報と顧客情報をマージ\n'

In [ ]:
merged_df = pd.read_csv("/content/drive/MyDrive/merged_df.csv", dtype={"article_id": str})

In [ ]:
articles.head()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,0108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,0108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,0108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,0110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,0110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


In [ ]:
articles.columns

Index(['article_id', 'product_code', 'prod_name', 'product_type_no',
       'product_type_name', 'product_group_name', 'graphical_appearance_no',
       'graphical_appearance_name', 'colour_group_code', 'colour_group_name',
       'perceived_colour_value_id', 'perceived_colour_value_name',
       'perceived_colour_master_id', 'perceived_colour_master_name',
       'department_no', 'department_name', 'index_code', 'index_name',
       'index_group_no', 'index_group_name', 'section_no', 'section_name',
       'garment_group_no', 'garment_group_name', 'detail_desc'],
      dtype='object')

In [ ]:
print(articles.department_no.unique())
print(articles.department_name.unique())

[1676 1339 3608 6515 1334 5883 2032 4342 4343 3419 6563 8559 8394 7956
 3611 3610 3937 7930 5283 3509 3929 4242 7988 1643 5963 8560 7668 7932
 5783 3709 5683 3946 8888 3519 5656 3945 5868 9985 4344 5962 1649 3209
 8615 1626 2031 5658 5961 1338 1722 7922 8768 7921 7931 3630 2035 5882
 3629 3710 8558 3948 3944 8756 1919 1939 5960 3705 7188 1745 1773 1779
 5878 7389 5828 1747 1772 1244 8956 3439 3409 9986 1344 1444 7920 6541
 1948 8757 4312 8917 3527 2950 1717 1343 5673 5957 8090 5687 5831 7388
 1666 5631 1920 8616 8617 5731 1949 2033 5672 8718 1201 5884 5958 1545
 1644 5679 6512 5686 3040 5956 1547 6564 1422 1670 4345 5832 5454 4225
 1641 1647 4214 4224 5952 9984 4310 4320 8316 8310 1636 5777 5555 2034
 6565 4210 4220 1414 1723 4314 5848 4317 1543 1510 1642 3529 4212 4213
 1909 8713 1646 3828 1322 6555 6525 5252 4215 8396 1313 7616 5690 6557
 8755 1610 3936 1410 4090 7656 1515 1640 7618 7648 1660 1710 3528 1912
 1941 7617 7613 1743 7020 3939 7812 1447 8716 8852 8812 3510 1926 7852
 5131 

In [ ]:
print(articles.perceived_colour_value_name.unique())
print(articles.perceived_colour_master_name.unique())

['Dark' 'Light' 'Dusty Light' 'Medium Dusty' 'Bright' 'Medium' 'Undefined'
 'Unknown']
['Black' 'White' 'Beige' 'Grey' 'Blue' 'Pink' 'Lilac Purple' 'Red' 'Mole'
 'Orange' 'Metal' 'Brown' 'Turquoise' 'Yellow' 'Khaki green' 'Green'
 'undefined' 'Unknown' 'Yellowish Green' 'Bluish Green']


In [ ]:
print(articles.graphical_appearance_no.unique())
print(articles.graphical_appearance_name.unique())

[1010016 1010017 1010001 1010010 1010019 1010011 1010002 1010023 1010005
 1010006 1010026 1010020 1010018 1010004 1010024 1010008 1010009 1010012
 1010014 1010013 1010027 1010007 1010021 1010022      -1 1010003 1010025
 1010028 1010015 1010029]
['Solid' 'Stripe' 'All over pattern' 'Melange' 'Transparent' 'Metallic'
 'Application/3D' 'Denim' 'Colour blocking' 'Dot' 'Other structure'
 'Contrast' 'Treatment' 'Check' 'Chambray' 'Front print'
 'Glittering/Metallic' 'Mixed solid/pattern' 'Placement print'
 'Other pattern' 'Neps' 'Embroidery' 'Lace' 'Jacquard' 'Unknown' 'Argyle'
 'Slub' 'Mesh' 'Sequin' 'Hologram']


In [ ]:
print(articles.section_no.unique())
print(articles.section_name.unique())

[16 61 62 44 26  8 66 22 46 51 79 31 65 52 60 58 72 55 21 25 23 15 27 47
  2 53 57 20 18 45 77 64 11 50 56 29 19 30 41  6 43  5 42 40 76 80 14 82
 49 24 28 97 70 71 48  4 17]
['Womens Everyday Basics' 'Womens Lingerie'
 'Womens Nightwear, Socks & Tigh' 'Baby Essentials & Complements'
 'Men Underwear' 'Mama' 'Womens Small accessories' 'Men H&M Sport'
 'Kids Boy' 'Divided Basics' 'Girls Underwear & Basics' 'Mens Outerwear'
 'Womens Big accessories' 'Divided Accessories'
 'Womens Swimwear, beachwear' 'Divided Selected' 'Boys Underwear & Basics'
 'Contemporary Street' 'Contemporary Casual' 'Men Accessories'
 'Men Suits & Tailoring' 'Womens Everyday Collection' 'Men Shoes'
 'Young Boy' 'H&M+' 'Divided Collection' 'Ladies Denim'
 'Contemporary Smart' 'Womens Trend' 'Kids Outerwear' 'Young Girl'
 'Womens Shoes' 'Womens Tailoring' 'Divided Projects' 'Denim Men'
 'Men Other' 'Womens Jackets' 'Men Other 2' 'Baby Boy' 'Womens Casual'
 'Kids Accessories, Swimwear & D' 'Ladies H&M Sport' 'Kids & Ba

In [ ]:
articles.isnull().sum()

article_id                        0
product_code                      0
prod_name                         0
product_type_no                   0
product_type_name                 0
product_group_name                0
graphical_appearance_no           0
graphical_appearance_name         0
colour_group_code                 0
colour_group_name                 0
perceived_colour_value_id         0
perceived_colour_value_name       0
perceived_colour_master_id        0
perceived_colour_master_name      0
department_no                     0
department_name                   0
index_code                        0
index_name                        0
index_group_no                    0
index_group_name                  0
section_no                        0
section_name                      0
garment_group_no                  0
garment_group_name                0
detail_desc                     416
dtype: int64

In [ ]:
articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105542 entries, 0 to 105541
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   article_id                    105542 non-null  int64 
 1   product_code                  105542 non-null  int64 
 2   prod_name                     105542 non-null  object
 3   product_type_no               105542 non-null  int64 
 4   product_type_name             105542 non-null  object
 5   product_group_name            105542 non-null  object
 6   graphical_appearance_no       105542 non-null  int64 
 7   graphical_appearance_name     105542 non-null  object
 8   colour_group_code             105542 non-null  int64 
 9   colour_group_name             105542 non-null  object
 10  perceived_colour_value_id     105542 non-null  int64 
 11  perceived_colour_value_name   105542 non-null  object
 12  perceived_colour_master_id    105542 non-null  int64 
 13 

In [ ]:
articles[["article_id", "section_name", "graphical_appearance_name", "perceived_colour_master_name", "department_name"]]

,article_id,section_name,graphical_appearance_name,perceived_colour_master_name,department_name
0,108775015,Womens Everyday Basics,Solid,Black,Jersey Basic
1,108775044,Womens Everyday Basics,Solid,White,Jersey Basic
2,108775051,Womens Everyday Basics,Stripe,White,Jersey Basic
3,110065001,Womens Lingerie,Solid,Black,Clean Lingerie
4,110065002,Womens Lingerie,Solid,White,Clean Lingerie
...,...,...,...,...,...
105537,953450001,Men Underwear,Placement print,Black,Socks Bin
105538,953763001,H&M+,Solid,Black,Jersey
105539,956217002,Womens Trend,Solid,Black,Jersey
105540,957375001,Divided Accessories,Solid,Black,Small Accessories


In [ ]:
# tmp = pd.merge(merged_df, articles[["article_id", "section_name", "graphical_appearance_name", "perceived_colour_master_name", "department_name"]], how="left", on="article_id")
tmp.to_csv("/content/drive/MyDrive/merged_df.csv", index=False)

In [ ]:
tmp.head()

,t_dat,customer_id,article_id,price,age,customer_sex,item_sex,mean_age,median_age,min_age,max_age,colour_group_name,section_name,graphical_appearance_name,perceived_colour_master_name,department_name
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0663713001,0.050831,24.0,woman,woman,32.998195,32.998195,19.0,68.0,Black,Womens Lingerie,Solid,Black,Expressive Lingerie
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0541518023,0.030492,24.0,woman,woman,34.475059,34.475059,17.0,68.0,Light Pink,Womens Lingerie,Solid,Pink,Casual Lingerie
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0505221004,0.015237,32.0,woman,devided,33.404762,33.404762,20.0,60.0,Pink,Divided Selected,Melange,Pink,Tops Knitwear DS
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687003,0.016932,32.0,woman,woman,35.399024,35.399024,18.0,90.0,Pink,Womens Everyday Collection,Melange,Pink,Campaigns
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687004,0.016932,32.0,woman,woman,37.326814,37.326814,18.0,80.0,Dark Green,Womens Everyday Collection,Melange,Green,Campaigns


In [ ]:
customers.head()

,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
customer_id,,,,,,
00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657,NaN,NaN,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
0000423b00ade91418cceaf3b26c6af3dd342b51fd051eec9c12fb36984420fa,NaN,NaN,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318,NaN,NaN,ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2c5feb1ca5dff07c43e,NaN,NaN,ACTIVE,NONE,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
00006413d8573cd20ed7128e53b7b13819fe5cfc2d801fe7fc0f26dd8d65a85a,1.0,1.0,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...


In [ ]:
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


In [ ]:
merged_df = pd.read_csv('merged_df.csv', dtype={"article_id": str})
merged_df.head()

,t_dat,customer_id,article_id,price,age,customer_sex,most_like_color,item_sex,mean_age,median_age,min_age,max_age
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0663713001,0.050831,24.0,woman,Black,woman,32.998195,32.998195,19.0,68.0
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0541518023,0.030492,24.0,woman,Black,woman,34.475059,34.475059,17.0,68.0
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0505221004,0.015237,32.0,woman,Black,devided,33.404762,33.404762,20.0,60.0
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687003,0.016932,32.0,woman,Black,woman,35.399024,35.399024,18.0,90.0
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687004,0.016932,32.0,woman,Black,woman,37.326814,37.326814,18.0,80.0


In [ ]:
# transactionデータの、9月売り上げデータ抜き出し
transactions_2018_9 = transactions.loc[transactions['t_dat'] <= '2018-09-30' ]
transactions_2019_9 = transactions.loc[('2019-09-20' <= transactions['t_dat']) & (transactions['t_dat'] <=  '2019-09-30')]
transactions_2020_9 = transactions.loc[transactions['t_dat'] >= '2020-09-01']
transaction_2018_2019 = pd.concat([transactions_2018_9, transactions_2019_9], axis=0)
transaction_9 = pd.concat([transaction_2018_2019, transactions_2020_9], axis=0)
# del transactions_2018_9, transactions_2019_9, transactions_2020_9
transaction_9.reset_index(drop=True)
# transaction_9.to_csv("transaction_9.csv")

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687004,0.016932,2
...,...,...,...,...,...
1934211,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,0929511001,0.059305,2
1934212,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,0891322004,0.042356,2
1934213,2020-09-22,fff380805474b287b05cb2a7507b9a013482f7dd0bce0e...,0918325001,0.043203,1
1934214,2020-09-22,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...,0833459002,0.006763,1


In [ ]:
# 2018年9月の売れ行き
sold_2018_9 = transactions_2018_9.groupby('article_id').count().sort_values('t_dat', ascending=False)\
                      .drop(['customer_id', 'price', 'sales_channel_id'], axis=1).rename(columns={'t_dat': 'sold_count'})
sold_2018_9['sold_ratio'] = sold_2018_9.sold_count / len(transactions_2018_9)

sold_num = 100
sold_2018_9_item = list(sold_2018_9[sold_2018_9.sold_count > sold_num].index)
df_2018 = articles[articles.article_id.isin(sold_2018_9_item)].reset_index(drop=True)

# sns.set_style("darkgrid")
# f, ax = plt.subplots(figsize=(10,5))
# ax = sns.histplot(data=df_2018, x='item_sex', color='orange')
# ax.set_xlabel('Distribution of item_sex')
# plt.show()

In [ ]:
# 2019年9月の売れ行き
sold_2019_9 = transactions_2019_9.groupby('article_id').count().sort_values('t_dat', ascending=False)\
                      .drop(['customer_id', 'price', 'sales_channel_id'], axis=1).rename(columns={'t_dat': 'sold_count'})
sold_2019_9['sold_ratio'] = sold_2019_9.sold_count / len(transactions_2019_9)

sold_num = 100
sold_2019_9_item = list(sold_2019_9[sold_2019_9.sold_count > sold_num].index)
df_2019 = articles[articles.article_id.isin(sold_2019_9_item)].reset_index(drop=True)

# sns.set_style("darkgrid")
# f, ax = plt.subplots(figsize=(10,5))
# ax = sns.histplot(data=df_2019, x='item_sex', color='orange')
# ax.set_xlabel('Distribution of item_sex')
# plt.show()

In [ ]:
# 2020年9月の売れ行き
sold_2020_9 = transactions_2020_9.groupby('article_id').count().sort_values('t_dat', ascending=False)\
                      .drop(['customer_id', 'price', 'sales_channel_id'], axis=1).rename(columns={'t_dat': 'sold_count'})
sold_2020_9['sold_ratio'] = sold_2020_9.sold_count / len(transactions_2020_9)

sold_num = 100
sold_2020_9_item = list(sold_2020_9[sold_2020_9.sold_count > sold_num].index)
df_2020 = articles[articles.article_id.isin(sold_2020_9_item)].reset_index(drop=True)

# sns.set_style("darkgrid")
# f, ax = plt.subplots(figsize=(10,5))
# ax = sns.histplot(data=df_2020, x='item_sex', color='orange')
# ax.set_xlabel('Distribution of item_sex')
# plt.show()

In [ ]:
# 全年の9月の売れ行き
df_all_9 = transaction_9.groupby('article_id').count().sort_values('t_dat', ascending=False)

In [ ]:
user_id = id[2]
user_bought = customer_buyitem.loc[user_id, 'article_id']
if isinstance(user_bought, float):
  # tmp = transaction_9.groupby('article_id').count().sort_values('t_dat', ascending=False)
  print('買上履歴なし')
  print(' '.join(list(df_all_9.index)[:12]))
else:
  # articles[articles.article_id.isin(user_bought)]
  user_sex = customer_buyitem.loc[user_id, 'sex']
  user_buy_price = customer_buyitem.loc[user_id, 'user_buy_min_max_price']
  user_like_color = customer_buyitem.loc[user_id, 'most_like_color']
  tmp = df_2020[(df_2020.item_sex == user_sex) & (df_2020.colour_group_name == user_like_color)].reset_index(drop=True)
  # if np.isnan(user_buy_price['mix']):
  #   tmp = df_2020[(df_2020.item_sex == user_sex) & (df_2020.colour_group_name == user_like_color)].reset_index(drop=True)
  # else:
  #   tmp = df_2020[(df_2020.item_sex == user_sex) & (df_2020.colour_group_name == user_like_color) &
  #         (df_2020.price > user_buy_price['mix']) & (df_2020.price < user_buy_price['max'])].reset_index(drop=True)
  tmp.article_id = tmp.article_id.astype("category")
  tmp.article_id.cat.set_categories(sold_2020_9_item, inplace=True)
  tmp.sort_values(['article_id'], inplace=True)
  print('買上履歴あり')
  print(f'性別：{user_sex}')
  print(f'買上履歴：{user_like_color}')
  print(f'買上履歴：{user_bought}')
  print(f'買上金額：{user_buy_price}')
  print(' '.join(list(tmp.article_id)[:12]))

In [ ]:
# sample_submission.csvに対して実施
def make_user_recommend(x):
  user_id = x
  user_bought = customer_buyitem.loc[user_id, 'article_id']
  # 買上履歴が存在しないユーザー
  if isinstance(user_bought, float):
    # tmp = transaction_9.groupby('article_id').count().sort_values('t_dat', ascending=False)
    # return ' '.join(list(df_all_9.index)[:12])
    return ' '
  # 買上履歴が存在するユーザー
  else:
    # articles[articles.article_id.isin(user_bought)]
    user_sex = customer_buyitem.loc[user_id, 'sex']
    user_buy_price = customer_buyitem.loc[user_id, 'user_buy_min_max_price']
    user_like_color = customer_buyitem.loc[user_id, 'most_like_color']
    tmp = df_2020[(df_2020.item_sex == user_sex) & (df_2020.colour_group_name == user_like_color)].reset_index(drop=True)
    # if np.isnan(user_buy_price['mix']):
    #   tmp = df_2020[(df_2020.item_sex == user_sex) & (df_2020.colour_group_name == user_like_color)].reset_index(drop=True)
    # else:
    #   tmp = df_2020[(df_2020.item_sex == user_sex) & (df_2020.colour_group_name == user_like_color) &
    #       (df_2020.price > user_buy_price['mix']) & (df_2020.price < user_buy_price['max'])].reset_index(drop=True)
    tmp.article_id = tmp.article_id.astype("category")
    tmp.article_id.cat.set_categories(sold_2020_9_item, inplace=True)
    tmp.sort_values(['article_id'], inplace=True)
    # del tmp
    return ' '.join(list(tmp.article_id)[:12])

In [ ]:
submission = pd.read_csv('sample_submission.csv')
submission['my_prediction'] = submission.customer_id.progress_apply(lambda x: make_user_recommend(x))

  0%|          | 0/1371980 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/pandas/core/arrays/categorical.py:2631: FutureWarning: The `inplace` parameter in pandas.Categorical.set_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  res = method(*args, **kwargs)
100%|██████████| 1371980/1371980 [1:29:51<00:00, 254.45it/s]


In [ ]:
new_submission = submission.set_index('customer_id').drop('prediction', axis=1).rename(columns={'my_prediction': 'prediction'})
# submission.to_csv('submission.csv')
new_submission

,prediction
customer_id,
00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657,0751471001 0915529003 0751471043 0863595006 06...
0000423b00ade91418cceaf3b26c6af3dd342b51fd051eec9c12fb36984420fa,0751471001 0915529003 0751471043 0863595006 06...
000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318,0751471001 0915529003 0751471043 0863595006 06...
00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2c5feb1ca5dff07c43e,0918292001 0852584001 0866731001 0905957001 09...
00006413d8573cd20ed7128e53b7b13819fe5cfc2d801fe7fc0f26dd8d65a85a,0751471001 0915529003 0751471043 0863595006 06...
...,...
ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e4747568cac33e8c541831,0751471001 0915529003 0751471043 0863595006 06...
ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab53481233731b5c4f8b7,0706016001 0715624001 0894780001 0797988002 08...
ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1778d0116cffd259264,0751471001 0915529003 0751471043 0863595006 06...


In [ ]:
# 予測に個性があるか確認
print(f'予測パターン数：{len(new_submission.prediction.unique())}')
print(f'1パターンあたりに予測したユーザー数：{len(customer_buyitem) / len(new_submission.prediction.unique())}')

予測パターン数：107
1パターンあたりに予測したユーザー数：12822.242990654206


In [ ]:
# predictionが空かどうか
count = 0
id = []
for i in list(new_submission.index):
  # print(type(new_submission.loc[i, 'prediction']))
  if isinstance(new_submission.loc[i, 'prediction'], float):
    count += 1
    id.append(i)

count


In [ ]:
new_submission.to_csv('submission.csv')

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

article = pd.read_csv("articles.csv", dtype={"article_id": str})

ohe_columns = []
total = 0

for col in article.columns:
    if article[col].dtype == "int64" and len(article[col].unique()) <= 500:
        ohe_columns.append(col)
        total += len(article[col].unique())
    
    print(col, article[col].dtype, len(article[col].unique()))
# print("Columns to use:", ohe_columns)

V = pd.get_dummies(article[ohe_columns], columns=ohe_columns).values

tfidf = TfidfVectorizer(min_df=3)
V_desc = tfidf.fit_transform(article["detail_desc"].fillna("nodesc"))

EMB_SIZE = 32

V = np.hstack([V.astype("float32"), V_desc.todense()])

svd = TruncatedSVD(n_components=EMB_SIZE, random_state=0)
svd.fit(V)
print("Explained variance ratio:", svd.explained_variance_ratio_.sum().item())

V = svd.transform(V)
print(V.shape)

tmp_article = article[["article_id"]]
tmp_article.reset_index(inplace=True)

tmp_v = pd.DataFrame(V)
tmp_v.reset_index(inplace=True)

tmp_con = pd.merge(tmp_article, tmp_v, how="left", on="index").drop(columns="index")

del article, tmp_article, tmp_v

from tqdm import tqdm

leave_num = -1
tmp_con["desc"] = "a"
for i in tqdm(range(len(tmp_con))):
  tmp = list(tmp_con.loc[i, 0:])
  tmp_con.at[i, "desc"] = tmp

tmp_con.drop(columns=list(tmp_con.columns)[1:-1], inplace=True)
tmp_con["desc"] = tmp_con["desc"].apply(lambda x: x[:leave_num])
tmp_con["desc_mean"] = tmp_con["desc"].progress_apply(lambda x: np.mean(x))
del tmp_con["desc"]

article_id object 105542
product_code int64 47224
prod_name object 45875
product_type_no int64 132
product_type_name object 131
product_group_name object 19
graphical_appearance_no int64 30
graphical_appearance_name object 30
colour_group_code int64 50
colour_group_name object 50
perceived_colour_value_id int64 8
perceived_colour_value_name object 8
perceived_colour_master_id int64 20
perceived_colour_master_name object 20
department_no int64 299
department_name object 250
index_code object 10
index_name object 10
index_group_no int64 5
index_group_name object 5
section_no int64 57
section_name object 56
garment_group_no int64 21
garment_group_name object 21
detail_desc object 43405
item_sex object 5
price float64 88089
mean_age float64 53108
median_age float64 53108
min_age float64 66
max_age float64 84


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


Explained variance ratio: 0.5510218832805773


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


(105542, 32)


100%|██████████| 105542/105542 [00:41<00:00, 2569.83it/s]


# wandb設定

In [ ]:
!pip install wandb -qqq
import wandb

     |████████████████████████████████| 1.8 MB 10.0 MB/s 
     |████████████████████████████████| 181 kB 51.9 MB/s 
     |████████████████████████████████| 144 kB 82.3 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 


In [ ]:
# Log in to your W&B account
wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# 以下kaggleノートブックのpytorchベースライン

<4/27実施すること><br>
次SEQ_LENの長さを変えてやってみる<br>
さらに直近の買い上げ履歴が存在しないユーザーについては、遡って購入履歴を取得して学習<br>
→transactionから、ユーザーが利用したことがあるかどうかを追跡確認<br>
エンコーディング手法も違うの試すとか確認する<br>



In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/

Mounted at /content/drive
/content/drive/MyDrive


In [ ]:
import numpy as np
import pandas as pd
import gc

# df_f = pd.read_csv("merged_df.csv", dtype={"article_id": str})
# n_articles = df_f["article_id"].nunique()
df = pd.read_csv("merged_df.csv", dtype={"article_id": str})
n_articles = df["article_id"].nunique()
print("number articles", n_articles)
# df = pd.merge(df_f, tmp_con, how="left", on="article_id")
# df["desc_mean"] = df["desc"].apply(lambda x: np.mean(x))
# del tmp_con, df_f
df.head()

number articles 104547


,t_dat,customer_id,article_id,price,age,customer_sex,item_sex,mean_age,median_age,min_age,max_age,colour_group_name,section_name,graphical_appearance_name,perceived_colour_master_name,department_name
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0663713001,0.050831,24.0,woman,woman,32.998195,32.998195,19.0,68.0,Black,Womens Lingerie,Solid,Black,Expressive Lingerie
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0541518023,0.030492,24.0,woman,woman,34.475059,34.475059,17.0,68.0,Light Pink,Womens Lingerie,Solid,Pink,Casual Lingerie
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0505221004,0.015237,32.0,woman,devided,33.404762,33.404762,20.0,60.0,Pink,Divided Selected,Melange,Pink,Tops Knitwear DS
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687003,0.016932,32.0,woman,woman,35.399024,35.399024,18.0,90.0,Pink,Womens Everyday Collection,Melange,Pink,Campaigns
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687004,0.016932,32.0,woman,woman,37.326814,37.326814,18.0,80.0,Dark Green,Womens Everyday Collection,Melange,Green,Campaigns


In [ ]:
df["t_dat"] = pd.to_datetime(df["t_dat"])
df["t_dat"].max()

Timestamp('2020-09-22 00:00:00')

In [ ]:
active_articles = df.groupby("article_id")["t_dat"].max().reset_index()
active_articles = active_articles[active_articles["t_dat"] >= "2019-09-01"].reset_index()
active_articles.shape

(72581, 3)

In [ ]:
df = df[df["article_id"].isin(active_articles["article_id"])].reset_index(drop=True)
df.shape

(29634404, 16)

In [ ]:
df["week"] = (df["t_dat"].max() - df["t_dat"]).dt.days // 7
df["week"].value_counts()

65     620104
13     549443
42     518403
12     517428
64     508664
        ...  
93     174190
102    164298
104    163143
97     162580
94     152807
Name: week, Length: 105, dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder


article_ids = np.concatenate([["placeholder"], np.unique(df["article_id"].values)])
customer_sex = np.concatenate([["placeholder"], np.unique(df["customer_sex"].values)])
item_sex = np.concatenate([["placeholder"], np.unique(df["item_sex"].values)])
item_color = np.concatenate([["placeholder"], np.unique(df["colour_group_name"].values)])
section_name = np.concatenate([["placeholder"], np.unique(df["section_name"].values)])
graphical_appearance_name = np.concatenate([["placeholder"], np.unique(df["graphical_appearance_name"].values)])
perceived_colour_master_name = np.concatenate([["placeholder"], np.unique(df["perceived_colour_master_name"].values)])
department_name = np.concatenate([["placeholder"], np.unique(df["department_name"].values)])

le_article = LabelEncoder()
le_customer_sex = LabelEncoder()
le_item_sex = LabelEncoder()
le_item_color = LabelEncoder()
le_section_name = LabelEncoder()
le_graphical_appearance_name = LabelEncoder()
le_perceived_colour_master_name = LabelEncoder()
le_department_name = LabelEncoder()

le_article.fit(article_ids)
le_customer_sex.fit(customer_sex)
le_item_sex.fit(item_sex)
le_item_color.fit(item_color)
le_section_name.fit(section_name)
le_graphical_appearance_name.fit(graphical_appearance_name)
le_perceived_colour_master_name.fit(perceived_colour_master_name)
le_department_name.fit(department_name)

df["article_id"] = le_article.transform(df["article_id"])
df["customer_sex"] = le_customer_sex.transform(df["customer_sex"])
df["item_sex"] = le_item_sex.transform(df["item_sex"])
df["colour_group_name"] = le_item_color.transform(df["colour_group_name"])
df["section_name"] = le_section_name.transform(df["section_name"])
df["graphical_appearance_name"] = le_graphical_appearance_name.transform(df["graphical_appearance_name"])
df["perceived_colour_master_name"] = le_perceived_colour_master_name.transform(df["perceived_colour_master_name"])
df["department_name"] = le_department_name.transform(df["department_name"])

del le_customer_sex, le_item_sex, le_item_color, le_section_name, le_graphical_appearance_name, le_perceived_colour_master_name, le_department_name
del customer_sex, item_sex, item_color, section_name, graphical_appearance_name, perceived_colour_master_name, department_name

In [ ]:
WEEK_HIST_MAX = 60

agg_param = {"article_id": list, 
                      "week": list, 
                      "mean_age": list, 
                      # "age": "min",
                      "price": list, 
                      "customer_sex": "min", 
                      "item_sex": list,
                      "colour_group_name": list,
                      # "desc_mean": list,
                      "section_name": list,
                      "graphical_appearance_name": list,
                      "perceived_colour_master_name": list,
                      "department_name": list,
             }

def create_dataset(df, week):
    hist_df = df[(df["week"] > week) & (df["week"] <= week + WEEK_HIST_MAX)]
    # 元のやつ
    # hist_df = hist_df.groupby("customer_id").agg({"article_id": list, "week": list}).reset_index()
    hist_df = hist_df.groupby("customer_id").agg(agg_param).reset_index()
    hist_df.rename(columns={"week": 'week_history'}, inplace=True)
    
    target_df = df[df["week"] == week]
    # 元のやつ
    # target_df = target_df.groupby("customer_id").agg({"article_id": list}).reset_index()
    target_df = target_df.groupby("customer_id").agg({"article_id": list}).reset_index()
    target_df.rename(columns={"article_id": "target"}, inplace=True)
    target_df["week"] = week
    result_df = target_df.merge(hist_df, on="customer_id", how="left")
    del hist_df, target_df
    gc.collect()
    return result_df

time = list(np.arange(1, 5))
v_week = 0
val_weeks = [v_week]
train_weeks = time


val_df = pd.concat([create_dataset(df, w) for w in val_weeks]).reset_index(drop=True)
train_df = pd.concat([create_dataset(df, w) for w in train_weeks]).reset_index(drop=True)
train_df.fillna({"customer_sex": 10000}, inplace=True)
val_df.fillna({"customer_sex": 10000}, inplace=True)

train_df.shape, val_df.shape

((300129, 14), (68984, 14))

In [ ]:
train_df.head()

,customer_id,target,week,article_id,week_history,mean_age,price,customer_sex,item_sex,colour_group_name,section_name,graphical_appearance_name,perceived_colour_master_name,department_name
0,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,[46435],1,"[29634, 898, 898, 30175, 30181, 61386, 59651, ...","[33, 33, 33, 24, 24, 22, 22, 22, 22, 22, 22, 2...","[33.62740788623142, 34.015625, 34.015625, 40.6...","[0.0254067796610169, 0.0220169491525423, 0.022...",4.0,"[5, 5, 5, 2, 2, 1, 5, 5, 5, 5, 5, 2, 2]","[1, 1, 1, 1, 17, 40, 0, 20, 20, 16, 16, 1, 16]","[51, 51, 51, 37, 37, 12, 52, 42, 42, 44, 44, 3...","[25, 25, 25, 25, 15, 25, 0, 21, 21, 26, 26, 25...","[1, 1, 1, 1, 6, 12, 0, 2, 2, 7, 7, 1, 7]","[191, 191, 191, 122, 122, 202, 173, 204, 204, ..."
1,0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...,"[28802, 1802]",1,"[27899, 43969]","[48, 48]","[31.561264822134387, 32.40611961057024]","[0.0254067796610169, 0.0220169491525423]",0.0,"[1, 1]","[0, 1]","[11, 11]","[25, 25]","[0, 1]","[27, 27]"
2,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,[31464],1,"[27215, 49253, 38278, 39876, 39876, 28839, 452...","[61, 61, 61, 61, 61, 59, 58, 58, 58, 58, 58, 5...","[34.41320754716981, 39.94542090656799, 41.1966...","[0.0169322033898305, 0.0423559322033898, 0.067...",4.0,"[5, 5, 1, 5, 5, 5, 4, 1, 1, 2, 5, 1, 4, 1, 1, ...","[1, 42, 2, 11, 11, 0, 5, 2, 2, 17, 20, 5, 1, 2...","[51, 44, 26, 44, 44, 52, 32, 26, 26, 38, 44, 2...","[25, 25, 7, 0, 0, 15, 25, 7, 7, 15, 7, 7, 25, ...","[1, 13, 2, 13, 13, 0, 2, 2, 2, 6, 2, 2, 1, 2, ...","[191, 173, 45, 54, 54, 122, 136, 46, 46, 28, 2..."
3,0003e867a930d0d6842f923d6ba7c9b77aba33fe2a0fbf...,"[14648, 11949, 66254, 66254, 67809]",1,"[54064, 1069, 64358, 31228, 65350, 44206, 6493...","[26, 26, 26, 12, 12, 7, 7, 7, 6, 6, 6, 2, 2, 2]","[34.50207468879668, 30.810420590081605, 33.191...","[0.0317627118644067, 0.008457627118644, 0.0338...",0.0,"[5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 25, 1, 25, 1, 1, 1]","[49, 11, 12, 13, 11, 12, 12, 12, 14, 12, 12, 1...","[25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 4, 10...","[1, 1, 1, 1, 1, 1, 1, 1, 8, 1, 8, 1, 1, 1]","[67, 27, 200, 59, 27, 174, 200, 200, 152, 198,..."
4,00040239317e877c77ac6e79df42eb2633ad38fcac09fc...,"[65340, 65341, 65340, 65341]",1,"[14710, 37767, 47997, 14709, 31788, 31788, 391...","[51, 51, 51, 51, 47, 47, 47, 23, 23, 23, 23]","[36.23604465709729, 35.18197879858657, 40.8967...","[0.0271016949152542, 0.0271016949152542, 0.027...",4.0,"[1, 1, 5, 1, 5, 5, 5, 1, 5, 5, 1]","[48, 1, 11, 1, 1, 1, 1, 16, 1, 47, 6]","[12, 12, 42, 12, 46, 46, 46, 11, 44, 52, 11]","[15, 25, 25, 25, 14, 14, 25, 25, 25, 25, 25]","[17, 1, 11, 1, 1, 1, 1, 7, 1, 16, 5]","[200, 200, 122, 200, 66, 66, 42, 27, 87, 33, 27]"


In [ ]:
val_df.head()

,customer_id,target,week,article_id,week_history,mean_age,price,customer_sex,item_sex,colour_group_name,section_name,graphical_appearance_name,perceived_colour_master_name,department_name
0,00039306476aaf41a07fed942884f16b30abfa83a2a8be...,[12382],0,"[37186, 42897, 1680, 60277, 18528, 18528, 2952...","[47, 47, 38, 38, 38, 38, 38, 34, 34, 32]","[34.65625, 34.75329341317365, 32.4739528599309...","[0.0338813559322033, 0.0338813559322033, 0.011...",4.0,"[5, 5, 5, 5, 5, 5, 5, 5, 5, 5]","[1, 1, 1, 14, 1, 1, 19, 24, 5, 5]","[41, 52, 47, 50, 47, 47, 46, 52, 52, 52]","[25, 25, 25, 25, 25, 25, 25, 25, 21, 21]","[1, 1, 1, 9, 1, 1, 0, 12, 2, 2]","[72, 173, 195, 90, 195, 195, 68, 173, 54, 54]"
1,0003e867a930d0d6842f923d6ba7c9b77aba33fe2a0fbf...,[55215],0,"[54064, 1069, 64358, 31228, 65350, 44206, 6493...","[26, 26, 26, 12, 12, 7, 7, 7, 6, 6, 6, 2, 2, 2...","[34.50207468879668, 30.810420590081605, 33.191...","[0.0317627118644067, 0.008457627118644, 0.0338...",0.0,"[5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 25, 1, 25, 1, 1, 1, 1...","[49, 11, 12, 13, 11, 12, 12, 12, 14, 12, 12, 1...","[25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 4, 10...","[1, 1, 1, 1, 1, 1, 1, 1, 8, 1, 8, 1, 1, 1, 1, ...","[67, 27, 200, 59, 27, 174, 200, 200, 152, 198,..."
2,000493dd9fc463df1acc2081450c9e75ef8e87d5dd17ed...,"[38205, 45221, 14652]",0,"[55736, 58768, 38205, 37629, 16955, 46441, 149...","[22, 22, 22, 22, 22, 22, 22, 19, 19, 19, 14, 1...","[36.34516129032258, 42.21164021164021, 32.8018...","[0.0406610169491525, 0.0423559322033898, 0.040...",0.0,"[5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 1, 5, 1, ...","[4, 1, 2, 42, 23, 47, 29, 20, 1, 20, 47, 40, 2...","[44, 12, 26, 12, 11, 15, 12, 12, 12, 12, 15, 4...","[25, 4, 7, 25, 26, 25, 1, 25, 25, 25, 0, 15, 7...","[0, 1, 2, 13, 11, 16, 16, 2, 1, 2, 16, 12, 12,...","[41, 56, 46, 200, 27, 57, 170, 206, 206, 206, ..."
3,000525e3fe01600d717da8423643a8303390a055c578ed...,[65083],0,NaN,NaN,NaN,NaN,10000.0,NaN,NaN,NaN,NaN,NaN,NaN
4,00077dbd5c4a4991e092e63893ccf29294a9d5c46e8501...,"[69861, 66066, 55, 63546, 71113, 72244, 71114,...",0,"[39570, 55016, 55984, 27642, 58332, 63709, 310...","[49, 37, 36, 36, 36, 36, 33, 33, 33, 33, 33, 3...","[35.38623595505618, 32.63486370157819, 31.9220...","[0.0525254237288135, 0.0338813559322033, 0.067...",4.0,"[5, 5, 5, 1, 1, 5, 1, 5, 5, 1, 5, 5, 5, 5, 5, ...","[1, 29, 19, 2, 29, 47, 22, 1, 1, 2, 5, 1, 1, 1...","[52, 44, 45, 26, 12, 50, 26, 41, 49, 12, 53, 4...","[25, 13, 25, 7, 25, 25, 7, 25, 25, 25, 25, 0, ...","[1, 16, 0, 2, 16, 16, 6, 1, 1, 2, 2, 5, 1, 1, ...","[204, 122, 149, 46, 200, 90, 46, 29, 35, 206, ..."


In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch
from tqdm import tqdm
import torch.nn as nn
import torch.nn.utils.rnn as rnn_utils
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

class HMDataset(Dataset):
    def __init__(self, df, seq_len, is_test=False):
        self.df = df.reset_index(drop=True)
        self.seq_len = seq_len
        self.is_test = is_test
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index):
        row = self.df.iloc[index]
        
        if self.is_test:
            target = torch.zeros(2).float()
        else:
            target = torch.zeros(len(article_ids)).float()
            for t in row.target:
                target[t] = 1.0
            
        article_hist = torch.zeros(self.seq_len).long()
        week_hist = torch.ones(self.seq_len).float()
        price_hist = torch.zeros(self.seq_len).float()
        mean_age_hist = torch.zeros(self.seq_len).float()
        # age_hist = torch.zeros(self.seq_len).float()
        customer_sex_hist = torch.zeros(self.seq_len).long()
        item_sex_hist = torch.zeros(self.seq_len).float()
        item_color_hist = torch.zeros(self.seq_len).float()
        # desc_hist = torch.zeros(self.seq_len).float()
        section_name_hist = torch.zeros(self.seq_len).float()
        graphical_appearance_name_hist = torch.zeros(self.seq_len).float()
        perceived_colour_master_name_hist = torch.zeros(self.seq_len).float()
        department_name_hist = torch.zeros(self.seq_len).float()
        
        if isinstance(row.article_id, list):
            if len(row.article_id) >= self.seq_len:
                article_hist = torch.LongTensor(row.article_id[-self.seq_len:])
                week_hist = (torch.LongTensor(row.week_history[-self.seq_len:]) - row.week)/WEEK_HIST_MAX/2
            else:
                article_hist[-len(row.article_id):] = torch.LongTensor(row.article_id)
                week_hist[-len(row.article_id):] = (torch.LongTensor(row.week_history) - row.week)/WEEK_HIST_MAX/2

        if isinstance(row.price, list):
            if len(row.price) >= self.seq_len:
                price_hist = torch.FloatTensor(row.price[-self.seq_len:])
            else:
                price_hist[-len(row.price):] = torch.FloatTensor(row.price)

        if isinstance(row.item_sex, list):
            if len(row.item_sex) >= self.seq_len:
                item_sex_hist = torch.FloatTensor(row.item_sex[-self.seq_len:])
            else:
                item_sex_hist[-len(row.item_sex):] = torch.FloatTensor(row.item_sex)

        if isinstance(row.mean_age, list):
            if len(row.mean_age) >= self.seq_len:
                mean_age_hist = torch.FloatTensor(row.mean_age[-self.seq_len:])
            else:
                mean_age_hist[-len(row.mean_age):] = torch.FloatTensor(row.mean_age)

        # if isinstance(row.age, list):
        #     if len(row.age) >= self.seq_len:
        #         age_hist = torch.FloatTensor(row.age[-self.seq_len:])
        #     else:
        #         age_hist[-len(row.age):] = torch.FloatTensor(row.age)

        if isinstance(row.colour_group_name, list):
            if len(row.colour_group_name) >= self.seq_len:
                item_color_hist = torch.FloatTensor(row.colour_group_name[-self.seq_len:])
            else:
                item_color_hist[-len(row.colour_group_name):] = torch.FloatTensor(row.colour_group_name)

        # if isinstance(row.desc_mean, list):
        #     if len(row.desc_mean) >= self.seq_len:
        #         desc_hist = torch.FloatTensor(row.desc_mean[-self.seq_len:])
        #     else:
        #         desc_hist[-len(row.desc_mean):] = torch.FloatTensor(row.desc_mean)

        if isinstance(row.section_name, list):
            if len(row.section_name) >= self.seq_len:
                section_name_hist = torch.FloatTensor(row.section_name[-self.seq_len:])
            else:
                section_name_hist[-len(row.colour_group_name):] = torch.FloatTensor(row.colour_group_name)

        if isinstance(row.graphical_appearance_name, list):
            if len(row.graphical_appearance_name) >= self.seq_len:
                graphical_appearance_name_hist = torch.FloatTensor(row.graphical_appearance_name[-self.seq_len:])
            else:
                graphical_appearance_name_hist[-len(row.graphical_appearance_name):] = torch.FloatTensor(row.graphical_appearance_name)

        if isinstance(row.perceived_colour_master_name, list):
            if len(row.perceived_colour_master_name) >= self.seq_len:
                perceived_colour_master_name_hist = torch.FloatTensor(row.perceived_colour_master_name[-self.seq_len:])
            else:
                perceived_colour_master_name_hist[-len(row.perceived_colour_master_name):] = torch.FloatTensor(row.perceived_colour_master_name)

        if isinstance(row.department_name, list):
            if len(row.department_name) >= self.seq_len:
                department_name_hist = torch.FloatTensor(row.department_name[-self.seq_len:])
            else:
                department_name_hist[-len(row.department_name):] = torch.FloatTensor(row.department_name)

        np_customer_sex = np.array(row.customer_sex).astype(np.float32)
        np_item_sex = np.array(row.item_sex).astype(np.float32)
        customer_sex_hist[-1] = torch.FloatTensor(np_customer_sex)
        # query相当：article_hist, week_hist, price_hist, mean_age_hist
        # q = torch.cat([article_hist.view(1, -1), week_hist.view(1, -1), price_hist.view(1, -1), mean_age_hist.view(1, -1)], dim=0)
        # memory相当：customer_sex_hist, item_sex_hist, item_color_hist
        # m = torch.cat([customer_sex_hist.view(1, -1), item_sex_hist.view(1, -1), item_color_hist.view(1, -1)], dim=0)
        # return q, m, target
      
        # 現時点のベストfeatures
        # return article_hist, week_hist, price_hist, customer_sex_hist, item_sex_hist, mean_age_hist, item_color_hist, target
        # テスト
        return article_hist, week_hist, price_hist, customer_sex_hist, item_sex_hist, mean_age_hist, item_color_hist,\
                  section_name_hist, graphical_appearance_name_hist, target
    
HMDataset(val_df, 16)[6]

(tensor([71112, 68883, 70515, 69487, 59095, 10784, 54486, 47410, 69762, 55981,
         67012, 39428, 71430, 71430, 72018, 72018]),
 tensor([0.0417, 0.0250, 0.0250, 0.0250, 0.0250, 0.0167, 0.0167, 0.0167, 0.0167,
         0.0167, 0.0083, 0.0083, 0.0083, 0.0083, 0.0083, 0.0083]),
 tensor([0.0339, 0.0491, 0.0409, 0.0981, 0.0424, 0.0163, 0.0652, 0.0244, 0.0488,
         0.0847, 0.0254, 0.0254, 0.0424, 0.0424, 0.0339, 0.0339]),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4]),
 tensor([5., 5., 5., 5., 1., 5., 5., 1., 1., 5., 5., 5., 5., 5., 5., 5.]),
 tensor([34.8939, 39.8159, 34.2375, 42.7899, 36.0000, 31.1812, 33.3196, 33.3624,
         32.6423, 37.3902, 33.6823, 36.8414, 38.6073, 38.6073, 42.3896, 42.3896]),
 tensor([ 0., 21., 47.,  8.,  6., 47.,  0.,  1., 24., 19., 40., 12., 15., 15.,
          1.,  1.]),
 tensor([44., 52., 52., 49., 14., 46., 44., 12., 14., 45., 44., 52., 44., 44.,
         52., 52.]),
 tensor([25., 21., 25., 25., 25., 25., 25., 25., 25., 25., 25., 25., 14., 

# Simple LSTM

In [ ]:
def adjust_lr(optimizer, epoch):
    if epoch < 1:
        lr = 5e-5
    elif epoch < 6:
        lr = 1e-3
    elif epoch < 9:
        lr = 1e-4
    else:
        lr = 1e-5

    for p in optimizer.param_groups:
        p['lr'] = lr
    return lr
    
def get_optimizer(net):
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()), lr=3e-4, betas=(0.9, 0.999),
                                 eps=1e-08)
    return optimizer

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class HMModel(nn.Module):
  # article_shape
    def __init__(self, article_shape):
        super(HMModel, self).__init__()
        
        self.article_emb = nn.Embedding(article_shape[0], embedding_dim=article_shape[1])
        
        self.article_likelihood = nn.Parameter(torch.zeros(article_shape[0]), requires_grad=True)
        self.top = nn.Sequential(
            nn.Conv1d(3, 32, kernel_size=1),
            nn.LeakyReLU(),
            nn.Conv1d(32, 8, kernel_size=1),
            nn.LeakyReLU(),
            nn.Conv1d(8, 1, kernel_size=1)
            )
        
    def forward(self, inputs):
        # self.article_emb = nn.Embedding(article_shape[0], embedding_dim=article_shape[1])
        # self.article_likelihood = nn.Parameter(torch.zeros(article_shape[0]), requires_grad=True)
        article_hist, week_hist = inputs[0], inputs[1]
        x = self.article_emb(article_hist)
        x = F.normalize(x, dim=2)
        
        x = x@F.normalize(self.article_emb.weight).T
        
        x, indices = x.max(axis=1)
        x = x.clamp(1e-3, 0.999)
        x = -torch.log(1/x - 1)
        
        max_week = week_hist.unsqueeze(2).repeat(1, 1, x.shape[-1]).gather(1, indices.unsqueeze(1).repeat(1, week_hist.shape[1], 1))
        max_week = max_week.mean(axis=1).unsqueeze(1)
        
        x = torch.cat([x.unsqueeze(1), max_week,
                       self.article_likelihood[None, None, :].repeat(x.shape[0], 1, 1)], axis=1)
        
        x = self.top(x).squeeze(1)
        return x
    
    
model = HMModel((len(le_article.classes_), 512))
# model = HMModel()
model = model.cuda()

In [ ]:
print(model)

HMModel(
  (article_emb): Embedding(72582, 512)
  (top): Sequential(
    (0): Conv1d(3, 32, kernel_size=(1,), stride=(1,))
    (1): LeakyReLU(negative_slope=0.01)
    (2): Conv1d(32, 8, kernel_size=(1,), stride=(1,))
    (3): LeakyReLU(negative_slope=0.01)
    (4): Conv1d(8, 1, kernel_size=(1,), stride=(1,))
  )
)


In [ ]:
!pip install torchinfo

In [ ]:
# from torchsummary import summary
from torchinfo import summary
summary(model)

Layer (type:depth-idx)                   Param #
HMModel                                  --
├─Embedding: 1-1                         37,161,984
├─Sequential: 1-2                        --
│    └─Conv1d: 2-1                       128
│    └─LeakyReLU: 2-2                    --
│    └─Conv1d: 2-3                       264
│    └─LeakyReLU: 2-4                    --
│    └─Conv1d: 2-5                       9
Total params: 37,162,385
Trainable params: 37,162,385
Non-trainable params: 0

In [ ]:
import sys

def calc_map(topk_preds, target_array, k=12):
    metric = []
    tp, fp = 0, 0
    
    for pred in topk_preds:
        if target_array[pred]:
            tp += 1
            metric.append(tp/(tp + fp))
        else:
            fp += 1
            
    return np.sum(metric) / min(k, target_array.sum())

def read_data(data):
    return tuple(d.cuda() for d in data[:-1]), data[-1].cuda()


def validate(model, val_loader, k=12):
    model.eval()
    
    tbar = tqdm(val_loader, file=sys.stdout)
    
    maps = []
    
    with torch.no_grad():
        for idx, data in enumerate(tbar):
            inputs, target = read_data(data)

            logits = model(inputs)

            _, indices = torch.topk(logits, k, dim=1)

            indices = indices.detach().cpu().numpy()
            target = target.detach().cpu().numpy()

            for i in range(indices.shape[0]):
                maps.append(calc_map(indices[i], target[i]))
        
    
    return np.mean(maps)

SEQ_LEN = 16

BS = 256
NW = 0

val_dataset = HMDataset(val_df, SEQ_LEN)
val_loader = DataLoader(val_dataset, batch_size=BS, shuffle=False, num_workers=NW,
                          pin_memory=False, drop_last=False)

In [ ]:
import gc
gc.collect()

179

In [ ]:
def dice_loss(y_pred, y_true):
    y_pred = y_pred.sigmoid()
    intersect = (y_true*y_pred).sum(axis=1)
    
    return 1 - (intersect/(intersect + y_true.sum(axis=1) + y_pred.sum(axis=1))).mean()


def train(model, train_loader, val_loader, epochs):
    np.random.seed(SEED)
    
    optimizer = get_optimizer(model)
    scaler = torch.cuda.amp.GradScaler()

    criterion = torch.nn.BCEWithLogitsLoss()
    torch.backends.cudnn.benchmark = True
    
    for e in range(epochs):
        model.train()
        tbar = tqdm(train_loader, file=sys.stdout)
        
        lr = adjust_lr(optimizer, e)
        
        loss_list = []

        for idx, data in enumerate(tbar):
            inputs, target = read_data(data)

            optimizer.zero_grad()
            
            with torch.cuda.amp.autocast():
                logits = model(inputs)
                loss = criterion(logits, target) + dice_loss(logits, target)
            
            
            #loss.backward()
            scaler.scale(loss).backward()
            #optimizer.step()
            scaler.step(optimizer)
            scaler.update()
            
            loss_list.append(loss.detach().cpu().item())
            
            avg_loss = np.round(100*np.mean(loss_list), 4)
            # wandb.log({"train_loss": avg_loss})

            tbar.set_description(f"Epoch {e+1} Loss: {avg_loss} lr: {lr}")
            
        val_map = validate(model, val_loader)
        # wandb.log({"valid_map": val_map})
        log_text = f"Epoch {e+1}\nTrain Loss: {avg_loss}\nValidation MAP: {val_map}\n"
            
        print(log_text)
        
        #logfile = open(f"models/{MODEL_NAME}_{SEED}.txt", 'a')
        #logfile.write(log_text)
        #logfile.close()
    return model

# wandbで可視化させるための前準備
# import wandb
import datetime

EPOCHS = 10
CONFIG = dict(
    batch_size_train = BS,
    batch_size_valid = BS,
    epochs = EPOCHS
)
MODEL_NAME = "exp005"
PROJECT_NAME = "HandM_{}_{}".format(MODEL_NAME, datetime.datetime.now().strftime('%Y-%m-%d'))
# wandb.init(project=PROJECT_NAME, entity="sasano_shouta")
# wandb.config = CONFIG
# wandbマジックコマンド
# wandb.watch(model, log_freq=1)

SEED = 0

train_dataset = HMDataset(train_df, SEQ_LEN)
train_loader = DataLoader(train_dataset, batch_size=BS, shuffle=True, num_workers=NW,
                          pin_memory=False, drop_last=True)

model = train(model, train_loader, val_loader, epochs=EPOCHS)

100%|██████████| 270/270 [02:03<00:00,  2.19it/s]
Epoch 1
Train Loss: 117.928
Validation MAP: 0.022606898415769475

100%|██████████| 270/270 [02:02<00:00,  2.21it/s]
Epoch 2
Train Loss: 100.1847
Validation MAP: 0.021160556975693173

100%|██████████| 270/270 [02:02<00:00,  2.20it/s]
Epoch 3
Train Loss: 99.295
Validation MAP: 0.023842645772259766

100%|██████████| 270/270 [02:01<00:00,  2.22it/s]
Epoch 4
Train Loss: 99.1948
Validation MAP: 0.023795965466379236

100%|██████████| 270/270 [02:01<00:00,  2.21it/s]
Epoch 5
Train Loss: 99.1465
Validation MAP: 0.023502504463419255

100%|██████████| 270/270 [02:01<00:00,  2.22it/s]
Epoch 6
Train Loss: 99.1096
Validation MAP: 0.023182801052101697

100%|██████████| 270/270 [02:01<00:00,  2.21it/s]
Epoch 7
Train Loss: 99.0643
Validation MAP: 0.022902443424410755

100%|██████████| 270/270 [02:00<00:00,  2.23it/s]
Epoch 8
Train Loss: 99.0588
Validation MAP: 0.022804928065373167

100%|██████████| 270/270 [02:01<00:00,  2.22it/s]
Epoch 9
Train Loss: 99

In [ ]:
train_dataset = HMDataset(train_df[train_df["week"] < time[-1]].append(val_df), SEQ_LEN)
train_loader = DataLoader(train_dataset, batch_size=BS, shuffle=True, num_workers=NW,
                          pin_memory=False, drop_last=True)

model = train(model, train_loader, val_loader, epochs=EPOCHS)

100%|██████████| 270/270 [02:00<00:00,  2.24it/s]
Epoch 1
Train Loss: 99.1016
Validation MAP: 0.022858175446374696

100%|██████████| 270/270 [02:01<00:00,  2.23it/s]
Epoch 2
Train Loss: 99.11
Validation MAP: 0.02491543264358883

100%|██████████| 270/270 [02:01<00:00,  2.23it/s]
Epoch 3
Train Loss: 99.0676
Validation MAP: 0.02546261537439651

100%|██████████| 270/270 [02:00<00:00,  2.25it/s]
Epoch 4
Train Loss: 99.0359
Validation MAP: 0.02602759640841539

100%|██████████| 270/270 [02:00<00:00,  2.25it/s]
Epoch 5
Train Loss: 99.0142
Validation MAP: 0.026183312801498826

100%|██████████| 270/270 [02:00<00:00,  2.24it/s]
Epoch 6
Train Loss: 98.9978
Validation MAP: 0.026324831100176457

100%|██████████| 270/270 [01:59<00:00,  2.26it/s]
Epoch 7
Train Loss: 98.9664
Validation MAP: 0.02631771118642902

100%|██████████| 270/270 [01:59<00:00,  2.25it/s]
Epoch 8
Train Loss: 98.963
Validation MAP: 0.026391385365403763

100%|██████████| 270/270 [02:00<00:00,  2.24it/s]
Epoch 9
Train Loss: 98.96
Val

In [ ]:
test_df = pd.read_csv('sample_submission.csv').drop("prediction", axis=1)
print(test_df.shape)
test_df.head()

(1371980, 1)


,customer_id
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...


In [ ]:
def create_test_dataset(test_df):
    week = -1
    test_df["week"] = week
    
    hist_df = df[(df["week"] > week) & (df["week"] <= week + WEEK_HIST_MAX)]
    hist_df = hist_df.groupby("customer_id").agg(agg_param).reset_index()
    hist_df.rename(columns={"week": 'week_history'}, inplace=True)
    
    
    return test_df.merge(hist_df, on="customer_id", how="left")

test_df = create_test_dataset(test_df)
# test_df.fillna({"customer_sex": 10000}, inplace=True)
test_df.head()

,customer_id,week,article_id,week_history,mean_age,price,customer_sex,item_sex,colour_group_name,section_name,graphical_appearance_name,perceived_colour_master_name,department_name
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,-1,"[16560, 47202, 47202, 44491, 22979, 44574, 514...","[52, 51, 51, 43, 42, 42, 42, 26, 26, 26, 26, 2...","[41.15816808309726, 37.67832167832168, 37.6783...","[0.0440508474576271, 0.0542203389830508, 0.054...",4.0,"[5, 5, 5, 0, 0, 5, 5, 5, 5, 1, 5, 5, 5]","[1, 1, 1, 1, 1, 1, 1, 19, 29, 16, 1, 1, 6]","[44, 52, 52, 55, 24, 52, 44, 44, 52, 12, 44, 4...","[25, 25, 25, 25, 25, 25, 0, 8, 8, 25, 25, 0, 21]","[1, 1, 1, 1, 1, 1, 1, 0, 16, 7, 1, 1, 5]","[204, 188, 188, 236, 220, 122, 54, 87, 81, 147..."
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,-1,"[10822, 31008, 14722, 37098, 31008, 57133, 631...","[58, 58, 58, 58, 56, 47, 35, 35, 35, 35, 35, 3...","[32.36363636363637, 31.655228758169933, 32.053...","[0.0220169491525423, 0.0677796610169491, 0.016...",4.0,"[1, 1, 1, 5, 1, 5, 5, 4, 5, 5, 5, 1, 5, 0, 5, ...","[29, 2, 1, 1, 2, 1, 1, 1, 5, 1, 1, 1, 20, 20, ...","[14, 26, 14, 46, 26, 44, 51, 32, 51, 51, 51, 1...","[0, 7, 0, 25, 7, 4, 21, 25, 0, 21, 21, 25, 25,...","[16, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 2, 2,...","[155, 46, 155, 66, 46, 189, 191, 135, 191, 191..."
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,-1,"[29634, 898, 898, 30175, 30181, 61386, 59651, ...","[33, 33, 33, 24, 24, 22, 22, 22, 22, 22, 22, 2...","[33.62740788623142, 34.015625, 34.015625, 40.6...","[0.0254067796610169, 0.0220169491525423, 0.022...",4.0,"[5, 5, 5, 2, 2, 1, 5, 5, 5, 5, 5, 2, 2, 4]","[1, 1, 1, 1, 17, 40, 0, 20, 20, 16, 16, 1, 16,...","[51, 51, 51, 37, 37, 12, 52, 42, 42, 44, 44, 3...","[25, 25, 25, 25, 15, 25, 0, 21, 21, 26, 26, 25...","[1, 1, 1, 1, 6, 12, 0, 2, 2, 7, 7, 1, 7, 7]","[191, 191, 191, 122, 122, 202, 173, 204, 204, ..."
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,-1,"[1274, 1274, 8932, 55319, 53046, 68897, 30567,...","[51, 49, 15, 15, 15, 5, 5, 5, 5]","[37.278362573099415, 37.278362573099415, 41.49...","[0.0444745762711864, 0.0593050847457627, 0.050...",4.0,"[5, 5, 5, 5, 5, 5, 4, 5, 4]","[1, 1, 23, 1, 1, 1, 1, 1, 15]","[45, 45, 46, 46, 46, 44, 27, 53, 27]","[7, 7, 25, 25, 25, 25, 16, 0, 25]","[1, 1, 11, 1, 1, 1, 1, 1, 6]","[149, 149, 68, 66, 68, 122, 126, 81, 127]"


In [ ]:
test_df["article_id"].isnull().mean()

0.2419226227787577

In [ ]:
test_ds = HMDataset(test_df, SEQ_LEN, is_test=True)
test_loader = DataLoader(test_ds, batch_size=BS, shuffle=False, num_workers=NW,
                          pin_memory=False, drop_last=False)


def inference(model, loader, k=12):
    model.eval()
    
    tbar = tqdm(loader, file=sys.stdout)
    
    preds = []
    
    with torch.no_grad():
        for idx, data in enumerate(tbar):
            inputs, target = read_data(data)

            logits = model(inputs)

            _, indices = torch.topk(logits, k, dim=1)

            indices = indices.detach().cpu().numpy()
            target = target.detach().cpu().numpy()

            for i in range(indices.shape[0]):
                preds.append(" ".join(list(le_article.inverse_transform(indices[i]))))
        
    
    return preds


test_df["prediction"] = inference(model, test_loader)

100%|██████████| 5360/5360 [1:06:08<00:00,  1.35it/s]


In [ ]:
# test_df.to_csv(f"submission{v_week}-{time[-1]}.csv", index=False, columns=["customer_id", "prediction"])
test_df.to_csv(f"submission.csv", index=False, columns=["customer_id", "prediction"])

# 作成済みのsubの内容を確認

In [ ]:
import pandas as pd
passed_sub = pd.read_csv('submission.csv', index_col=0)
# predictionが空かどうか
count = 0
id = []
length = []
for i in list(passed_sub.index):
  # print(type(new_submission.loc[i, 'prediction']))
  length.append(len(passed_sub.loc[i, 'prediction']))
  if isinstance(passed_sub.loc[i, 'prediction'], float):
    count += 1
    id.append(i)

print(f"predictがNANである個数： {count}")
print(f"predictの個数： {list(set(length))}")
# 予測に個性があるか確認
print(f'予測パターン数：{len(passed_sub.prediction.unique())}')
print(f'1パターンあたりに予測したユーザー数：{len(passed_sub) / len(passed_sub.prediction.unique())}')

predictがNANである個数： 0
predictの個数： [131]
予測パターン数：227604
1パターンあたりに予測したユーザー数：6.02792569550623


# kaggleへsubする

In [2]:
# kaggleのAPIを使えるようにする
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 64 bytes


In [3]:
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

Found existing installation: kaggle 1.5.12
Uninstalling kaggle-1.5.12:
  Successfully uninstalled kaggle-1.5.12
     |████████████████████████████████| 2.1 MB 14.3 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.2/58.2 KB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72858 sha256=257a4eb0202f491fb14e8e97d721cc6f8a1b7c0d1c888023e932a229ffc554fc
  Stored in directory: /root/.cache/pip/wheels/aa/e7/e7/eb3c3d514c33294d77ddd5a856bdd58dc9c1fabbed59a02a2b
Successfully built kaggle


In [4]:
!kaggle competitions submit -c h-and-m-personalized-fashion-recommendations -f submission.csv -m test

100% 258M/258M [00:08<00:00, 32.8MB/s]
Successfully submitted to H&M Personalized Fashion Recommendations

# 以下Notionとの連携

In [ ]:
import warnings
warnings.simplefilter('ignore')

# Notionとの連携に使用
import datetime
import json
from pprint import pprint
import requests

import numpy as np
import pandas as pd

from myNotion.config import Config
from myNotion.propertytype import PropertyType
from myNotion.to_notion_function import to_notion, Cell

In [ ]:
response = requests.request('GET', url=f'{Config.notion_endpoint}/databases/{Config.database_id}', headers=Config.headers)
# pprint(response.json())
properties = {}
for k, v in response.json()['properties'].items():
    properties[k] = v['type']

MODEL_NAME = "exp009"
# pprint(properties)
cells = [
    Cell('Submit', properties, 'True'),
    Cell('Exp', properties, MODEL_NAME),
    Cell('Date', properties, datetime.datetime.now().strftime('%Y-%m-%d')),
    Cell('Base Exp', properties, MODEL_NAME),
    Cell('CV', properties, 0.02651244869297067),
    Cell('Public LB', properties, 0.0184),
    Cell('Model Name', properties, 'nn.Sequential'),
    Cell('Preprocessing', properties, ['mean_age', 'price', 'item_sex', 'item_color', 'customer_sex', "section_name", "graphical_appearance_name", "perceived_colour_master_name", "department_name"]),
    Cell('Memo', properties, 'exp004に特徴量追加。SEQ_LEN=16, BS=256')
]

response = to_notion(cells).json()
pprint(response)

{'archived': False,
 'cover': None,
 'created_by': {'id': '06fd336e-24b9-44b9-93ee-a07e07c6a6e5', 'object': 'user'},
 'created_time': '2022-04-26T15:04:00.000Z',
 'icon': None,
 'id': '06c2584b-fb3a-490b-95e7-e8621b5a031c',
 'last_edited_by': {'id': '06fd336e-24b9-44b9-93ee-a07e07c6a6e5',
                    'object': 'user'},
 'last_edited_time': '2022-04-26T15:04:00.000Z',
 'object': 'page',
 'parent': {'database_id': '9bebc48f-0782-4bb5-8957-e4edb97340cf',
            'type': 'database_id'},
 'properties': {'Base Exp': {'id': 'r%60%7BZ',
                             'rich_text': [{'annotations': {'bold': False,
                                                            'code': False,
                                                            'color': 'default',
                                                            'italic': False,
                                                            'strikethrough': False,
                                                            'u